<h1>Color Detection</h1>

<h2>imports</h2>

In [2]:
import numpy as np
import cv2
from PIL import Image

<h2>helper functions</h2>

In [1]:
# Function to get HSV limits for a given color
def get_limits(color):
    c = np.uint8([[color]])  # BGR values
    hsvC = cv2.cvtColor(c, cv2.COLOR_BGR2HSV)

    hue = hsvC[0][0][0]  # Get the hue value

    # Handle red hue wrap-around
    if hue >= 165:  # Upper limit for divided red hue
        lowerLimit = np.array([hue - 10, 100, 100], dtype=np.uint8)
        upperLimit = np.array([180, 255, 255], dtype=np.uint8)
    elif hue <= 15:  # Lower limit for divided red hue
        lowerLimit = np.array([0, 100, 100], dtype=np.uint8)
        upperLimit = np.array([hue + 10, 255, 255], dtype=np.uint8)
    else:
        lowerLimit = np.array([hue - 10, 100, 100], dtype=np.uint8)
        upperLimit = np.array([hue + 10, 255, 255], dtype=np.uint8)

    return lowerLimit, upperLimit

<h2> webcam </h2>

In [3]:
# Initialize video capture
cap = cv2.VideoCapture(1)

# Define a yellow color in BGR format
yellow = [255, 255, 0]

while True:
    # Read a frame from the video
    ret, frame = cap.read()
    
    # Convert the frame to HSV format
    hsv_image = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    
    # Get HSV limits for the specified color (yellow)
    lower_limit, upper_limit = get_limits(color=yellow)
    
    # Create a binary mask based on color limits
    mask = cv2.inRange(hsv_image, lower_limit, upper_limit)
    
    # Convert the mask to an Image
    mask_ = Image.fromarray(mask)
    
    # Get the bounding box of the detected region in the mask
    bbox = mask_.getbbox()
    
    # If a bounding box is found, draw a green rectangle around the region in the original frame
    if bbox is not None:
        x1, y1, x2, y2 = bbox
        frame = cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 5)
    
    # Display the frame
    cv2.imshow('frame', frame)
    
    # Break the loop if the 'q' key is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release video capture resources
cap.release()

# Close all OpenCV windows
cv2.destroyAllWindows()